In [1]:
import  datalab.storage as storage
import pandas as pd
from io import BytesIO
import numpy as np

In [ ]:
for i in range(1, 4):
  for j in range(1, 5):
    pre = "Base" +str(i)+str(j)
    bucket = storage.Bucket('bmi260-dr')
    data = bucket.item('blood_info_{0}.csv'.format(pre))
    uri = data.uri
    %gcs read --object $uri --variable data
    img_data = pd.read_csv(BytesIO(data))
    label_data = bucket.item('{0}/Annotation_{1}.csv'.format(pre, pre))
    uri = label_data.uri
    %gcs read --object $uri --variable data2
    label_df = pd.read_csv(BytesIO(data2))
    print(list(label_df.columns.values))
    label_df = label_df[['Image name', 'Retinopathy grade']]
    merged = label_df.merge(img_data, left_on='Image name', right_on='IMG', how='inner')
    dataset = [i for k in range(merged.shape[0])]
    merged['dataset'] = dataset
    bucket.item('blood_info_{}_with_label.csv'.format(pre)).write_to(merged.to_csv(),'text/csv')